In [1]:
# 環境構築

!apt-get update
!apt-get install -y libsndfile1-dev
!pip install scipy
!pip install scikit-learn
!pip install librosa

/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/a

In [2]:
# clips 作成

SAMPLING_RATE = 44100 # 変更不可

clip_size = 81920 # 楽曲を再構築するパーツ1つあたりの大きさ
step_size = 20480 # clip をずらすときの大きさ
window_size = 10240 # CQT_CHROMA を取得するのに使用するサンプル数
hop_size = 640 # window をずらすときの大きさ

from scipy.io.wavfile import read, write
import glob
import os
import numpy as np
import librosa
import sys

if os.path.isfile("../data/out/clips4predict.wav"):
    print("../data/out/clips4predict.wav exists.")
    files = ["../data/out/clips4predict.wav"]
else:
    # clips.npy をもとにデータを作成する 
    if os.path.isfile("../data/arrays/clips4predict.npy"):
        print("loading ../data/arrays/clips4predict.npy ...")
        clips = np.load("../data/arrays/clips4predict.npy")
        print("creating ../data/out/clips4predict.wav ...")
        write("../data/out/clips4predict.wav", SAMPLING_RATE, clips.reshape((clips.shape[0] * clips.shape[1], )))
        files = ["../data/out/clips4predict.wav"]
        
    # clips,npy がないとき /data/wav44100 内の WAV ファイルを参照する
    else:
        files = glob.glob("../data/wav4predict/*.wav")
        #files = ["../data/wav44100/3DEmbodimentFromLines.wav"] # デバッグ用 
        
if len(files) == 0:
    print("WAV ファイルが見つかりませんでした。")
    sys.exit(1)
        
clips_filename = "../data/arrays/c" + str(clip_size) + "_s" + str(step_size) + "_f32_clips4predict"

if os.path.isfile(clips_filename + ".npy"):
    print("loading " + clips_filename + ".npy ...")
    clips = np.load(clips_filename + ".npy")
else:
    print("creating " + clips_filename + ".npy ...")
    raw_data_list = [librosa.load(file, sr=SAMPLING_RATE)[0] for file in files] # 左の音だけ使う

    clips = np.zeros((0, clip_size), dtype=np.float32)
    for raw_data in raw_data_list:
        tmp = [raw_data[i:i + clip_size] for i in range(0, len(raw_data) - clip_size, step_size)]
        clips = np.vstack((clips, np.array(tmp)))
    np.save(clips_filename, clips)

print("The clip array has " + str(clips.shape[0]) + " clips.")

loading ../data/arrays/c81920_s20480_f32_clips4predict.npy ...
The clip array has 41637 clips.


In [3]:
# cqt 作成

n_bins = 84

cqt_filename = clips_filename + "_w" + str(window_size) + "_h" + str(hop_size) + "_cqts4predict"

if os.path.isfile(cqt_filename + ".npy"):
    print("loading " + cqt_filename + ".npy ...")
    cqts = np.load(cqt_filename + ".npy")
else:
    print("creating " + cqt_filename + ".npy ...")
    num = 0
    for clip in clips:
        print("cqt progress: clip " + str(num + 1) + " / " + str(len(clips)))
        
        tmp_cqt = librosa.cqt(clip, sr=SAMPLING_RATE, hop_length=hop_size, n_bins=n_bins)
        tmp_cqt = tmp_cqt.reshape((1, tmp_cqt.shape[0], tmp_cqt.shape[1]))
        
        if num == 0:
            cqts = tmp_cqt
        else:
            cqts = np.vstack((cqts, tmp_cqt))
            
        num += 1

    np.save(cqt_filename, cqts)
    
print("The cqt array has " + str(cqts.shape[0]) + " cqts.")
print("cqts.shape: " + str(cqts.shape))
print("Type(cqts[0][0][0]): " + str(type(cqts[0][0][0])))
print("np.max(cqts[0][0]): " + str(np.max(cqts[0][0])))

loading ../data/arrays/c81920_s20480_f32_clips4predict_w10240_h640_cqts4predict.npy ...
The cqt array has 41637 cqts.
cqts.shape: (41637, 84, 129)
Type(cqts[0][0][0]): <class 'numpy.complex64'>
np.max(cqts[0][0]): (0.04157509+0.022495579j)


In [4]:
# cqt_choroma 作成

def Preprocessing(array):
    array = np.abs(array)
    array = np.log(array + 1)
    array = array / np.log(np.finfo(np.float32).max)
    array = array.T
    return array

cqt_chroma_filename = clips_filename + "_w" + str(window_size) + "_h" + str(hop_size) + "_cqt_chromas4predict"

if os.path.isfile(cqt_chroma_filename + ".npy"):
    print("loading " + cqt_chroma_filename + ".npy ...")
    cqt_chromas= np.load(cqt_chroma_filename + ".npy")
else:
    print("creating " + cqt_chroma_filename + ".npy ...")
    num = 0
    for cqt in cqts:
        print("cqt_chroma progress: clip " + str(num + 1) + " / " + str(len(cqts)))
        
        tmp_cqt_chroma = librosa.feature.chroma_cqt(C=cqt, sr=SAMPLING_RATE)
        tmp_cqt_chroma = Preprocessing(tmp_cqt_chroma)
        tmp_cqt_chroma = tmp_cqt_chroma.reshape((1, tmp_cqt_chroma.shape[0], tmp_cqt_chroma.shape[1]))
        
        if num == 0:
            cqt_chromas = tmp_cqt_chroma
        else:
            cqt_chromas = np.vstack((cqt_chromas, tmp_cqt_chroma))
            
        num += 1

    np.save(cqt_chroma_filename, cqt_chromas)
    
print("The cqt_chroma array has " + str(cqt_chromas.shape[0]) + " cqt_chromas.")
print("cqt_chromas.shape: " + str(cqt_chromas.shape)) # clip 番号、window 番号、 chroma 番号 になる
print("Type(cqt_chromas[0][0][0]): " + str(type(cqt_chromas[0][0][0])))
print("np.max(cqt_chromas[0][0]): " + str(np.max(cqt_chromas[0][0])))

loading ../data/arrays/c81920_s20480_f32_clips4predict_w10240_h640_cqt_chromas4predict.npy ...
The cqt_chroma array has 41637 cqt_chromas.
cqt_chromas.shape: (41637, 129, 12)
Type(cqt_chromas[0][0][0]): <class 'numpy.float32'>
np.max(cqt_chromas[0][0]): 0.0078125


In [5]:
# max_norm 作成

num_divide = 8

max_norm_filename = clips_filename + "_d" + str(num_divide) + "_max_norms4predict"

if os.path.isfile(max_norm_filename + ".npy"):
    print("loading " + max_norm_filename + ".npy ...")
    max_norms= np.load(max_norm_filename + ".npy")
else:
    print("creating " + max_norm_filename + ".npy ...")
    num = 0
    for clip in clips:
        print("max_norm progress: clip " + str(num + 1) + " / " + str(len(clips)))
        max_norm = [np.max(clip[int((i / num_divide) * len(clip)):int(((i + 1) / num_divide) * len(clip))] ** 2) for i in range(num_divide)]
    
        if num == 0:
            max_norms = np.array(max_norm)
        else:
            max_norms = np.vstack((max_norms, np.array(max_norm)))
        num += 1
        
    np.save(max_norm_filename, max_norms)
    
print("The max_norm array has " + str(max_norms.shape[0]) + " max_norms.")
print("max_norms.shape: " + str(max_norms.shape))
print("Type(max_norms[0][0]): " + str(type(max_norms[0][0])))
print("np.max(max_norms[0]): " + str(np.max(max_norms[0])))

loading ../data/arrays/c81920_s20480_f32_clips4predict_d8_max_norms4predict.npy ...
The max_norm array has 41637 max_norms.
max_norms.shape: (41637, 8)
Type(max_norms[0][0]): <class 'numpy.float32'>
np.max(max_norms[0]): 0.14306161


In [6]:
# 学習データとテストデータに分ける

def normalize(array):
    if (np.sum(array) == 0):
        return array
    else:
        return array / np.sum(array)
    
cqt_chroma_sum_threshold = 0.01 # 次の window の sum が閾値に満たないときに除外します
test_data_rate = 0.1

window_num_per_clip = cqt_chromas.shape[1]

cqt_chromas = np.array([np.hstack((cqt_chromas[i], np.repeat(np.array([max_norms[i]]), cqt_chromas.shape[1], axis=0))) for i in range(len(max_norms))])
cqt_chromas = np.concatenate([cqt_chromas[:-1, :, :], cqt_chromas[1:, 0, :].reshape(cqt_chromas.shape[0] - 1, 1, cqt_chromas.shape[2])], 1)

p = np.random.permutation(len(cqt_chromas))
cqt_chromas = cqt_chromas[p]
clips = clips[p]

x = np.delete(cqt_chromas, np.where(np.sum(cqt_chromas[:,-1,:], axis=1) < cqt_chroma_sum_threshold)[0], axis=0)

x_test = x[:int(x.shape[0] * test_data_rate), :window_num_per_clip, :].reshape(int(x.shape[0] * test_data_rate), window_num_per_clip, x.shape[2], 1)
x_train = x[int(x.shape[0] * test_data_rate):, :window_num_per_clip, :].reshape(x.shape[0] - int(x.shape[0] * test_data_rate), window_num_per_clip, x.shape[2], 1)

y = np.array([x[:, window_num_per_clip, i] for i in range(x.shape[2])])
y_maxs = np.max(y, axis=1)
y_mins = np.min(y, axis=1)

y_tests =  np.array([(x[:int(x.shape[0] * test_data_rate), window_num_per_clip, i] - y_mins[i]) / (y_maxs[i] - y_mins[i]) for i in range(x.shape[2])])
y_trains = np.array([(x[int(x.shape[0] * test_data_rate):, window_num_per_clip, i] - y_mins[i]) / (y_maxs[i] - y_mins[i]) for i in range(x.shape[2])])

print("x_train.shape: " + str(x_train.shape))
print("x_test.shape: " + str(x_test.shape))
print("y_trains.shape: " + str(y_trains.shape))
print("y_tests.shape: " + str(y_tests.shape))

x_train.shape: (37439, 129, 20, 1)
x_test.shape: (4159, 129, 20, 1)
y_trains.shape: (20, 37439)
y_tests.shape: (20, 4159)


In [8]:
# 楽曲の出力

import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from keras.models import load_model
import soundfile as sf

epochs = 200
batch_size = 64
data_name = "c" + str(clip_size) + "_s" + str(step_size) + "_w" + str(window_size) + "_h" + str(hop_size) + "_d" + str(num_divide)

def add_fade(x, ft_len):
    r = np.arange(0, ft_len)*np.pi/ft_len
    w_fo = (0.5+0.5*np.cos(r))**0.5
    w_fi = (0.5-0.5*np.cos(r))**0.5
    
    x[0:ft_len]        *= w_fi
    x[clip_size-ft_len::] *= w_fo
    return x

def gen_xfade(x_pre, x_next, ft_len):
    if x_pre is None:
        xfade = x_next
    else:
        x_pre_begin = x_pre[:x_pre.shape[0] - ft_len]
        x_pre_end = x_pre[x_pre.shape[0] - ft_len:]
        x_pre_len = clip_size
        x_next_len = clip_size
        x_pre_len -= ft_len
        x_next_len -= ft_len
        xfade = np.concatenate((x_pre_begin, np.concatenate((x_pre_end, np.zeros(x_next_len))) + x_next))
    return xfade

def create_music_consider_weights(fname, rw):
    first_index = np.random.randint(0, len(cqt_chromas))
    predict_index = first_index
    out = None
    fade_samples = 10240
    
    for i in range(num_clips):
        print("-- generate " + str(i + 1) + " / " + str(num_clips))
        predict_vec = np.zeros((0, ))
        for i in range(x.shape[2]):
            predict = models[i].predict(np.array([cqt_chromas[predict_index, :-1]]))[0] * (y_maxs[i] - y_mins[i]) + y_mins[i]
            predict_vec = np.hstack((predict_vec, predict))
        predict_chroma = predict_vec[:len(predict_vec)-num_divide]
        predict_rhythm = predict_vec[len(predict_vec)-num_divide:]

        index = 0
        similar_index = 0
        score = -1 - rw
        chroma_score = 0
        rhythm_score = 0
        for cqt_chroma in cqt_chromas[:, (int)(fade_samples / hop_size)]:
            cqt_chroma_chroma = cqt_chroma[:len(cqt_chroma)-num_divide]
            cqt_chroma_rhythm = cqt_chroma[len(cqt_chroma)-num_divide:]
            tmp_chroma_score = cosine_similarity(np.array([predict_chroma]), np.array([cqt_chroma_chroma]))
            tmp_rhythm_score = cosine_similarity(np.array([predict_rhythm]), np.array([cqt_chroma_rhythm]))
            tmp_score = tmp_chroma_score + rw * tmp_rhythm_score
            if tmp_score > score:
                score = tmp_score
                similar_index = index
            index += 1

        print("score: " + str(score))
        print("predict_vec: " + str(predict_vec))
        print("cqt_chromas[similar_index]: " + str(cqt_chromas[similar_index, 0]))
        print("similar_index: " + str(similar_index))
        print("--")

        predict_index = similar_index

        tmp = add_fade(clips[predict_index], fade_samples)
        out = gen_xfade(out, tmp, fade_samples)
        
    sf.write(fname, out, SAMPLING_RATE, subtype="PCM_16")

models = []
for i in range(x.shape[2]):
    model_name = "independent_" + data_name + "_batch" + str(batch_size) + "_e" + str(epochs) + "_" + str(i)
    
    models.append(load_model("../data/models/" + model_name))

num_clips = 30

for i in range(7):
    rhythm_weight = (4.0 + i) / (10.0 - i)
    file_name = "out_" + "independent_" + data_name + "_batch" + str(batch_size) + "_e" + str(epochs) + "_rw" + format(rhythm_weight, '.2f').replace('.', '_') + "_PREDICT"
    fname = "../data/out/" + file_name + "_track" + str(i) + ".wav"
    print("creating " + fname + " ...")
    create_music_consider_weights(fname, rhythm_weight)

creating ../data/out/out_independent_c81920_s20480_w10240_h640_d8_batch64_e200_rw0_40_PREDICT_track0.wav ...
-- generate 1 / 30
1/1 [==============================] - 0s 36ms/step
score: [[1.28781415]]
predict_vec: [0.00524771 0.00552676 0.00080557 0.00066392 0.00257091 0.00412765
 0.00296001 0.00224862 0.00385826 0.00293695 0.00178831 0.00044142
 0.04771885 0.09171733 0.10641351 0.17612453 0.05846683 0.04805848
 0.07396504 0.06673148]
cqt_chromas[similar_index]: [0.00731779 0.         0.0078125  0.         0.         0.
 0.         0.00419624 0.00409454 0.         0.         0.
 0.07679265 0.04678281 0.04810558 0.1095366  0.08697891 0.07092189
 0.06108899 0.05543346]
similar_index: 28058
--
-- generate 2 / 30
1/1 [==============================] - 0s 16ms/step
score: [[1.32657763]]
predict_vec: [0.00399213 0.00138321 0.00205129 0.00098285 0.00266845 0.00091715
 0.00673214 0.00168118 0.00202056 0.00074888 0.0031226  0.00320504
 0.02687767 0.060702   0.0436168  0.03960459 0.03187332 0.0

1/1 [==============================] - 0s 16ms/step
score: [[1.33580137]]
predict_vec: [0.00145209 0.00315241 0.00057    0.00030613 0.00070816 0.00060517
 0.00179932 0.00388811 0.00338087 0.00140781 0.00270401 0.00044681
 0.06697349 0.08499457 0.04976185 0.07905758 0.05172578 0.06942417
 0.06010871 0.05730277]
cqt_chromas[similar_index]: [0.0078125  0.         0.00158702 0.00572499 0.00541132 0.
 0.00336936 0.00341922 0.00419461 0.00163199 0.00381044 0.
 0.12870112 0.17636001 0.12816523 0.19187383 0.11922597 0.16846606
 0.11732632 0.12446675]
similar_index: 8548
--
-- generate 7 / 30
1/1 [==============================] - 0s 15ms/step
score: [[1.31915961]]
predict_vec: [0.00058833 0.00196341 0.00308763 0.00608216 0.00233464 0.00039959
 0.00075948 0.00189774 0.0034934  0.00482478 0.00474028 0.00094574
 0.06933647 0.10530782 0.05973934 0.09400318 0.06165327 0.0727038
 0.07620835 0.07243954]
cqt_chromas[similar_index]: [0.0027385  0.         0.00595336 0.00439806 0.         0.00209388
 0.

1/1 [==============================] - 0s 16ms/step
score: [[1.33112379]]
predict_vec: [0.00270675 0.0033677  0.0015031  0.00315367 0.00266282 0.00040018
 0.00220408 0.00458468 0.00135689 0.00022462 0.00290152 0.00751011
 0.08686604 0.05068923 0.04419342 0.04219536 0.05709911 0.06644317
 0.06077326 0.05368792]
cqt_chromas[similar_index]: [0.         0.00176532 0.00275309 0.         0.00546484 0.00543112
 0.         0.0078125  0.00249494 0.00473114 0.00611403 0.
 0.0922496  0.07078379 0.06098343 0.09855476 0.10061565 0.07192506
 0.09887117 0.09114989]
similar_index: 23514
--
-- generate 12 / 30
1/1 [==============================] - 0s 16ms/step
score: [[1.30792964]]
predict_vec: [0.0020771  0.00178288 0.0013279  0.00055032 0.00164757 0.00094654
 0.00206776 0.00360442 0.00494718 0.00093396 0.00189432 0.00667629
 0.03199348 0.05475805 0.0505257  0.04052136 0.05214738 0.05068438
 0.05610475 0.0501462 ]
cqt_chromas[similar_index]: [0.00718503 0.0078125  0.00566322 0.         0.         0.


1/1 [==============================] - 0s 17ms/step
score: [[1.32383556]]
predict_vec: [0.00091615 0.00239263 0.00265952 0.00571271 0.00538445 0.00393863
 0.00134678 0.00049004 0.00292795 0.00448125 0.00219988 0.00147419
 0.01715168 0.03214334 0.04543492 0.06710925 0.05703622 0.06111966
 0.05721673 0.05378428]
cqt_chromas[similar_index]: [0.0078125  0.         0.         0.         0.         0.00194767
 0.00353163 0.00252774 0.00294513 0.         0.00301371 0.00250605
 0.04395554 0.08309025 0.04125977 0.10399286 0.05433247 0.07447629
 0.04781819 0.07056475]
similar_index: 38375
--
-- generate 17 / 30
1/1 [==============================] - 0s 14ms/step
score: [[1.3341724]]
predict_vec: [0.00538159 0.00166373 0.00566081 0.00322251 0.00400403 0.00049247
 0.00090238 0.00070399 0.0024894  0.00092559 0.00163749 0.00096266
 0.02394485 0.05774805 0.02720477 0.0422955  0.02533464 0.03996581
 0.04291555 0.03870368]
cqt_chromas[similar_index]: [0.         0.         0.00655974 0.         0.     

1/1 [==============================] - 0s 16ms/step
score: [[1.35660681]]
predict_vec: [0.00113414 0.00505579 0.0008467  0.00098103 0.00055612 0.00369218
 0.00715273 0.00105542 0.001459   0.0006209  0.00068828 0.00011212
 0.03012507 0.03663561 0.04261809 0.04131921 0.03008088 0.035282
 0.03715421 0.03887369]
cqt_chromas[similar_index]: [0.         0.0078125  0.         0.         0.00089231 0.00078521
 0.         0.         0.         0.         0.         0.
 0.12269661 0.15041415 0.2025934  0.24259412 0.22994202 0.14165686
 0.11915222 0.16909292]
similar_index: 4763
--
-- generate 22 / 30
1/1 [==============================] - 0s 22ms/step
score: [[1.32545549]]
predict_vec: [0.0051165  0.00155575 0.00093794 0.00403681 0.001931   0.00049719
 0.00093971 0.00186755 0.00089215 0.00070068 0.00090057 0.00193671
 0.11142527 0.13464014 0.11864724 0.07964493 0.06256549 0.09559026
 0.07491672 0.08311984]
cqt_chromas[similar_index]: [0.         0.00743113 0.00440427 0.00364476 0.         0.
 0.

...

1/1 [==============================] - 0s 19ms/step
score: [[3.39941921]]
predict_vec: [0.00640825 0.00326899 0.00135988 0.00071198 0.00328183 0.00150966
 0.0019785  0.00045134 0.0005427  0.0009811  0.00368113 0.00673479
 0.03064547 0.03535348 0.03364829 0.03392882 0.04824622 0.03865413
 0.04123428 0.04036236]
cqt_chromas[similar_index]: [0.         0.         0.         0.         0.0032757  0.
 0.         0.         0.0078125  0.         0.         0.00541683
 0.10462366 0.14142723 0.0890613  0.07391104 0.10971847 0.12372497
 0.13682865 0.1095467 ]
similar_index: 17728
--
-- generate 8 / 30
1/1 [==============================] - 0s 21ms/step
score: [[3.38710754]]
predict_vec: [0.00291948 0.00396053 0.00153121 0.00192117 0.00180678 0.00054016
 0.00453275 0.00641042 0.00240559 0.00112957 0.00240645 0.0012948
 0.04813256 0.04328615 0.05492061 0.06930298 0.07397522 0.06144891
 0.06567399 0.05977201]
cqt_chromas[similar_index]: [0.00456512 0.         0.         0.         0.         0.
 0

1/1 [==============================] - 0s 17ms/step
score: [[3.37149275]]
predict_vec: [0.00164793 0.00569579 0.00029985 0.00343824 0.00310802 0.00080941
 0.00377156 0.00174719 0.0006995  0.00156003 0.00302239 0.00116801
 0.02051329 0.06148251 0.0445791  0.03098415 0.04512011 0.0425878
 0.04238781 0.03898818]
cqt_chromas[similar_index]: [0.         0.0078125  0.00650989 0.00523428 0.         0.
 0.         0.         0.00127998 0.         0.         0.00749452
 0.05406959 0.11854202 0.10371749 0.08272119 0.09056117 0.08423776
 0.10375681 0.09775164]
similar_index: 26775
--
-- generate 13 / 30
1/1 [==============================] - 0s 17ms/step
score: [[3.40403888]]
predict_vec: [5.42020518e-03 6.58837287e-03 1.45740272e-03 1.22100837e-03
 9.80241573e-04 9.81154037e-04 2.88509368e-03 6.09594281e-04
 1.30169501e-04 4.35888884e-04 8.28356206e-05 1.61338481e-03
 5.67421392e-02 4.71831970e-02 4.59447615e-02 4.70877141e-02
 5.45136668e-02 5.42735308e-02 5.41777723e-02 5.07681817e-02]
cqt_chr

1/1 [==============================] - 0s 18ms/step
score: [[3.35810569]]
predict_vec: [0.00217056 0.00342748 0.00070769 0.00048875 0.00218691 0.00429637
 0.00295425 0.0020206  0.00216091 0.00326961 0.00101654 0.00052455
 0.11019625 0.05693137 0.06961702 0.09877384 0.12962802 0.14193246
 0.1116455  0.13526624]
cqt_chromas[similar_index]: [0.0078125  0.00645495 0.00436306 0.         0.00114711 0.00335663
 0.00379868 0.00406874 0.00390414 0.         0.         0.00591585
 0.07803231 0.07810906 0.06125504 0.05792536 0.16914313 0.15312481
 0.13735972 0.17210546]
similar_index: 16847
--
-- generate 18 / 30
1/1 [==============================] - 0s 18ms/step
score: [[3.34777528]]
predict_vec: [4.26685018e-03 2.69436929e-03 1.15877879e-03 8.22905393e-04
 4.02321899e-03 2.17209244e-03 9.66967316e-04 5.15607535e-04
 1.59463799e-03 4.11672611e-03 5.63991314e-04 2.61079222e-05
 3.21369581e-02 3.23972255e-02 8.56579989e-02 8.56862366e-02
 7.39393532e-02 9.67867821e-02 9.29143056e-02 8.35211501e-02

1/1 [==============================] - 0s 18ms/step
score: [[3.3629751]]
predict_vec: [0.0064956  0.00379889 0.00052902 0.00135454 0.0035639  0.00086663
 0.00147284 0.00293886 0.0014182  0.00143307 0.00305659 0.00245028
 0.00943725 0.01347869 0.0110219  0.0174719  0.01593772 0.0128639
 0.01686164 0.02440168]
cqt_chromas[similar_index]: [0.         0.0078125  0.         0.00709946 0.         0.
 0.         0.00126927 0.         0.         0.00223112 0.00175836
 0.04045171 0.04474604 0.06269852 0.09438441 0.08297592 0.09623098
 0.10164437 0.1180487 ]
similar_index: 11714
--
-- generate 23 / 30
1/1 [==============================] - 0s 19ms/step
score: [[3.41240106]]
predict_vec: [0.00338573 0.00200754 0.00037564 0.00272001 0.00142636 0.00074731
 0.00088806 0.00555123 0.00156453 0.00124463 0.00048581 0.00037565
 0.03292283 0.05260386 0.04117501 0.05297951 0.05328363 0.067207
 0.054394   0.05309976]
cqt_chromas[similar_index]: [0.         0.00536695 0.         0.         0.         0.
 0. 

1/1 [==============================] - 0s 18ms/step
score: [[3.35483531]]
predict_vec: [0.0029975  0.00149477 0.00329844 0.00270966 0.00494472 0.00156573
 0.00083327 0.00222433 0.00071674 0.00198596 0.00167707 0.00102925
 0.01496995 0.0477093  0.0215098  0.05541476 0.04662707 0.0789483
 0.05843845 0.0507036 ]
cqt_chromas[similar_index]: [0.00163425 0.00285183 0.0038992  0.0078125  0.         0.
 0.0028471  0.00300484 0.         0.00369535 0.00091247 0.00252955
 0.03357233 0.0557933  0.04412205 0.10889114 0.08683497 0.09773256
 0.10594063 0.07491834]
similar_index: 28618
--
-- generate 28 / 30
1/1 [==============================] - 0s 18ms/step
score: [[3.35642892]]
predict_vec: [1.69246850e-04 3.17864725e-03 2.15543620e-03 1.70241343e-03
 3.75994644e-03 8.61626293e-04 3.40158516e-03 1.23827846e-03
 1.60357892e-03 1.37809967e-03 7.25889532e-03 1.78018054e-05
 2.52004471e-02 6.02444746e-02 4.33908217e-02 5.39533533e-02
 5.55451959e-02 4.21576947e-02 4.71465923e-02 3.68538238e-02]
cqt_chr